## In this notebook:

* 3 input dataframe: chr18, 19, 20
* using all numeric scores for predictors
* fully connected neural net
* training with dataloading and epochs
* aucuracy assessement of .954 on held-out test 

### Imports

In [13]:
#!pip install numpy 
#!pip install pandas
#!pip install torch
#!pip install sklearn

import numpy as np
import pandas as pd
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from torch.utils.data import DataLoader, Dataset
from torch.nn import functional as F
from sklearn.metrics import accuracy_score
from torch.autograd import Variable
import torch.optim as optim

### Manipulating data
* Import csv we want
* Map our features to usuable integers
* Map our labels to a one hot encoding

In [14]:
#################################################### 
## pull in pickled dataframe:
df = pd.read_pickle("./pickled_df/all_scores_chr18,20.pkl")


# Split our data into two separate sets for training purposes
train, test = train_test_split(df, shuffle=False)

print(test.head())
print(train.shape, test.shape)

                     SIFT_score SIFT4G_score Polyphen2_HDIV_score  \
chr pos      ref alt                                                
20  63408436 G   A            0          0.0                    0   
    63408468 G   A            0        0.382                    0   
    63408484 T   A            0            0                    0   
    63408517 G   A            0        0.002                    0   
    63408524 G   C            0        0.002                    0   

                     Polyphen2_HVAR_score LRT_score MutationTaster_score  \
chr pos      ref alt                                                       
20  63408436 G   A                      0  0.000016              0.99997   
    63408468 G   A                      0  0.040548              0.97061   
    63408484 T   A                      0  0.114895                    1   
    63408517 G   A                      0  0.744725             0.998933   
    63408524 G   C                      0  0.663984         

### DataLoader
* Implement the Dataset class supplied by PyTorch
  * Used Map style
  * API: https://pytorch.org/docs/stable/data.html#map-style-datasets
* Instanciate DataLoader using Dataset instance

In [15]:
class DataSetdbNSFP(Dataset):
    def __init__(self, data):
        self.data = torch.FloatTensor(data.values.astype('float'))
        
    def __len__(self):
        return len(self.data)
    def __getitem__(self, index):
            target = self.data[index][-1]
            data_val = self.data[index] [:-1]
            return data_val,target
        
train_dataset = DataSetdbNSFP(train)
valid_dataset = DataSetdbNSFP(train)
test_dataset = DataSetdbNSFP(test)

train_loader = DataLoader(train_dataset, batch_size=59, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=59, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=59, shuffle=True)


### Model
* This model is a simple implementation
* Based on https://github.com/ieee8023/NeuralNetwork-Examples/blob/master/pytorch/pytorch-mnist.ipynb

In [16]:

class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        # In: nb of features, out: nb of examples
        self.fc = nn.Linear(27, 1000)
        # In: nb of examples, out: nb of predictions
        self.fc2 = nn.Linear(1000, 2)

    def forward(self, x):
        x = x.view((-1, 27))
        h = F.relu(self.fc(x))
        h = self.fc2(h)
        # Softmax to get the actual labels
        return F.softmax(h, dim=1)  ## Note: can potentially use this to get confidence of category predicted 
    
model = Model()
# if cuda:
#     model.cuda()

# Optimizer based on Adam algorithm, uses a slightly lower rate
optimizer = optim.Adam(model.parameters(), lr=1e-4)

### Training
* Run a fixed number of epochs
* After each epoch, run the validation set to adjust learning
* Print results
* Based on example: https://github.com/ieee8023/NeuralNetwork-Examples/blob/master/pytorch/pytorch-mnist.ipynb

In [17]:
EPOCHS = 20

model.train()
for epoch in range(EPOCHS):
    for batch_idx, (data, target) in enumerate(train_loader):

        data, target = Variable(data), Variable(target)
        print(data.shape)
        print(target.shape)
        
#         if cuda:
#             data, target = data.cuda(), target.cuda()
        
        optimizer.zero_grad()
        y_pred = model(data) 
        
        # Calculate and appy loss
        loss = F.cross_entropy(y_pred, target.long())
        loss.backward()
        optimizer.step()

        print('\r Train Epoch: {}/{} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
            epoch+1,
            EPOCHS,
            batch_idx * len(data), 
            len(train_loader.dataset),
            100. * batch_idx / len(train_loader), 
            loss.cpu().data.item()))
    
    # NOTE: The Variable class is a wrapper class around a tensor with the added functionalities of back propagation
    # For x, take all cols of datafram except the last one
    evaluate_x = Variable(valid_loader.dataset.data[:, 0:-1].type_as(torch.FloatTensor()))
    print(evaluate_x)
    # For y, take last col
    evaluate_y = Variable(valid_loader.dataset.data[:, -1:])
#     if cuda:
#         evaluate_x, evaluate_y = evaluate_x.cuda(), evaluate_y.cuda()
    model.eval()
    output = model(evaluate_x)
    pred = output.data.max(1)[1]
    
    y_labels = torch.flatten(evaluate_y.data)
    print('Predictions:', pred)
    print('Actual values:', y_labels)
    d = pred.eq(y_labels).cpu()
    accuracy = d.sum().item()*1./d.size()[0]
    
    print('\r Train Epoch: {}/{} [{}/{} ({:.0f}%)]\tLoss: {:.6f}\t Validation Accuracy: {:.2f}%'.format(
        epoch+1,
        EPOCHS,
        len(train_loader.dataset), 
        len(train_loader.dataset),
        100. * batch_idx / len(train_loader), 
        loss.cpu().data.item(),
        accuracy*100))

torch.Size([59, 27])
torch.Size([59])
 Train Epoch: 1/20 [0/1050 (0%)]	Loss: 0.691467
torch.Size([59, 27])
torch.Size([59])
 Train Epoch: 1/20 [59/1050 (6%)]	Loss: 0.715688
torch.Size([59, 27])
torch.Size([59])
 Train Epoch: 1/20 [118/1050 (11%)]	Loss: 0.714008
torch.Size([59, 27])
torch.Size([59])
 Train Epoch: 1/20 [177/1050 (17%)]	Loss: 0.705688
torch.Size([59, 27])
torch.Size([59])
 Train Epoch: 1/20 [236/1050 (22%)]	Loss: 0.704686
torch.Size([59, 27])
torch.Size([59])
 Train Epoch: 1/20 [295/1050 (28%)]	Loss: 0.700236
torch.Size([59, 27])
torch.Size([59])
 Train Epoch: 1/20 [354/1050 (33%)]	Loss: 0.681440
torch.Size([59, 27])
torch.Size([59])
 Train Epoch: 1/20 [413/1050 (39%)]	Loss: 0.691327
torch.Size([59, 27])
torch.Size([59])
 Train Epoch: 1/20 [472/1050 (44%)]	Loss: 0.668031
torch.Size([59, 27])
torch.Size([59])
 Train Epoch: 1/20 [531/1050 (50%)]	Loss: 0.659719
torch.Size([59, 27])
torch.Size([59])
 Train Epoch: 1/20 [590/1050 (56%)]	Loss: 0.664678
torch.Size([59, 27])
torch

torch.Size([59, 27])
torch.Size([59])
 Train Epoch: 5/20 [531/1050 (50%)]	Loss: 0.544102
torch.Size([59, 27])
torch.Size([59])
 Train Epoch: 5/20 [590/1050 (56%)]	Loss: 0.581701
torch.Size([59, 27])
torch.Size([59])
 Train Epoch: 5/20 [649/1050 (61%)]	Loss: 0.572254
torch.Size([59, 27])
torch.Size([59])
 Train Epoch: 5/20 [708/1050 (67%)]	Loss: 0.533179
torch.Size([59, 27])
torch.Size([59])
 Train Epoch: 5/20 [767/1050 (72%)]	Loss: 0.565750
torch.Size([59, 27])
torch.Size([59])
 Train Epoch: 5/20 [826/1050 (78%)]	Loss: 0.592929
torch.Size([59, 27])
torch.Size([59])
 Train Epoch: 5/20 [885/1050 (83%)]	Loss: 0.549200
torch.Size([59, 27])
torch.Size([59])
 Train Epoch: 5/20 [944/1050 (89%)]	Loss: 0.581006
torch.Size([47, 27])
torch.Size([47])
 Train Epoch: 5/20 [799/1050 (94%)]	Loss: 0.567255
tensor([[0.0000, 0.0000, 0.0000,  ..., 0.7248, 0.6453, 0.6356],
        [0.0000, 0.0000, 0.0000,  ..., 0.7025, 0.0000, 0.7144],
        [0.0080, 0.0130, 0.5320,  ..., 0.6979, 0.5705, 0.7144],
       

torch.Size([59, 27])
torch.Size([59])
 Train Epoch: 10/20 [177/1050 (17%)]	Loss: 0.503263
torch.Size([59, 27])
torch.Size([59])
 Train Epoch: 10/20 [236/1050 (22%)]	Loss: 0.539627
torch.Size([59, 27])
torch.Size([59])
 Train Epoch: 10/20 [295/1050 (28%)]	Loss: 0.504693
torch.Size([59, 27])
torch.Size([59])
 Train Epoch: 10/20 [354/1050 (33%)]	Loss: 0.508165
torch.Size([59, 27])
torch.Size([59])
 Train Epoch: 10/20 [413/1050 (39%)]	Loss: 0.531695
torch.Size([59, 27])
torch.Size([59])
 Train Epoch: 10/20 [472/1050 (44%)]	Loss: 0.510588
torch.Size([59, 27])
torch.Size([59])
 Train Epoch: 10/20 [531/1050 (50%)]	Loss: 0.483626
torch.Size([59, 27])
torch.Size([59])
 Train Epoch: 10/20 [590/1050 (56%)]	Loss: 0.498369
torch.Size([59, 27])
torch.Size([59])
 Train Epoch: 10/20 [649/1050 (61%)]	Loss: 0.538039
torch.Size([59, 27])
torch.Size([59])
 Train Epoch: 10/20 [708/1050 (67%)]	Loss: 0.470859
torch.Size([59, 27])
torch.Size([59])
 Train Epoch: 10/20 [767/1050 (72%)]	Loss: 0.485821
torch.Size

torch.Size([59, 27])
torch.Size([59])
 Train Epoch: 14/20 [295/1050 (28%)]	Loss: 0.439232
torch.Size([59, 27])
torch.Size([59])
 Train Epoch: 14/20 [354/1050 (33%)]	Loss: 0.505108
torch.Size([59, 27])
torch.Size([59])
 Train Epoch: 14/20 [413/1050 (39%)]	Loss: 0.428515
torch.Size([59, 27])
torch.Size([59])
 Train Epoch: 14/20 [472/1050 (44%)]	Loss: 0.478776
torch.Size([59, 27])
torch.Size([59])
 Train Epoch: 14/20 [531/1050 (50%)]	Loss: 0.445186
torch.Size([59, 27])
torch.Size([59])
 Train Epoch: 14/20 [590/1050 (56%)]	Loss: 0.511199
torch.Size([59, 27])
torch.Size([59])
 Train Epoch: 14/20 [649/1050 (61%)]	Loss: 0.486223
torch.Size([59, 27])
torch.Size([59])
 Train Epoch: 14/20 [708/1050 (67%)]	Loss: 0.438194
torch.Size([59, 27])
torch.Size([59])
 Train Epoch: 14/20 [767/1050 (72%)]	Loss: 0.456845
torch.Size([59, 27])
torch.Size([59])
 Train Epoch: 14/20 [826/1050 (78%)]	Loss: 0.480266
torch.Size([59, 27])
torch.Size([59])
 Train Epoch: 14/20 [885/1050 (83%)]	Loss: 0.441013
torch.Size

 Train Epoch: 18/20 [799/1050 (94%)]	Loss: 0.444863
tensor([[0.0000, 0.0000, 0.0000,  ..., 0.7248, 0.6453, 0.6356],
        [0.0000, 0.0000, 0.0000,  ..., 0.7025, 0.0000, 0.7144],
        [0.0080, 0.0130, 0.5320,  ..., 0.6979, 0.5705, 0.7144],
        ...,
        [0.0000, 0.1150, 0.0000,  ..., 0.5473, 0.6453, 0.5628],
        [0.0000, 0.0000, 0.0000,  ..., 0.5473, 0.6453, 0.5628],
        [0.0000, 0.0000, 0.0000,  ..., 0.0606, 0.0742, 0.0792]])
Predictions: tensor([1, 1, 0,  ..., 1, 1, 1])
Actual values: tensor([1., 1., 0.,  ..., 1., 1., 1.])
 Train Epoch: 18/20 [1050/1050 (94%)]	Loss: 0.444863	 Validation Accuracy: 90.29%
torch.Size([59, 27])
torch.Size([59])
 Train Epoch: 19/20 [0/1050 (0%)]	Loss: 0.492994
torch.Size([59, 27])
torch.Size([59])
 Train Epoch: 19/20 [59/1050 (6%)]	Loss: 0.422735
torch.Size([59, 27])
torch.Size([59])
 Train Epoch: 19/20 [118/1050 (11%)]	Loss: 0.442952
torch.Size([59, 27])
torch.Size([59])
 Train Epoch: 19/20 [177/1050 (17%)]	Loss: 0.457394
torch.Size([5

### Testing
* Take testing dataset, run it against current model
* Evaluate testing accuracy

In [18]:
# Follow same steps as when we test with validation set, except with the test set
evaluate_x = Variable(test_loader.dataset.data[:, 0:-1].type_as(torch.FloatTensor()))
evaluate_y = Variable(test_loader.dataset.data[:, -1:])

model.eval()
output = model(evaluate_x)
pred = output.data.max(1)[1]

y_labels = torch.flatten(evaluate_y.data)
d = pred.eq(y_labels).cpu()
accuracy = d.sum().item()*1./d.size()[0]  ## Accuracy = (TP+TN)/(TP+TN+FP+FN)

print('\n Testing Accuracy: {:.2f}%'.format(

    accuracy*100))


 Testing Accuracy: 95.44%
